# Function Calling Agent - Demo Notebook

## Overview

Azure OpenAI Function Callingの動作検証ノートブック。

**Day 17学習目標**:
- Function Calling基礎理解
- ツール定義スキーマ作成
- 並列実行（Parallel Function Calling）
- エラーハンドリング

## 1. Setup

In [ ]:
import sys
sys.path.append('/home/claude')

from app.agents.tools.tool_definitions import tool_registry
from app.agents.tools.implementations import (
    calculate_impl,
    get_current_datetime_impl,
    get_equipment_status_impl
)
from app.agents.function_calling_agent_mock import MockFunctionCallingAgent

import json
from IPython.display import display, Markdown, JSON

## 2. Tool Definitions

Function Callingで使用するツールスキーマを確認。

In [ ]:
# 全ツールスキーマ取得
schemas = tool_registry.get_all_tool_schemas()

print(f"Available Tools: {len(schemas)}\n")

for schema in schemas:
    name = schema['function']['name']
    desc = schema['function']['description']
    print(f"📌 {name}")
    print(f"   {desc}\n")

## 3. Individual Tool Testing

各ツールの単体動作確認。

In [ ]:
# Test 1: Calculator
print("=== Calculator Tool ===")

test_expressions = [
    "sqrt(16) + 2 ** 3",
    "pi * 2",
    "sin(pi / 2)"
]

for expr in test_expressions:
    result = calculate_impl(expr)
    print(f"{expr} = {result['result']}")

In [ ]:
# Test 2: DateTime
print("=== DateTime Tool ===")

result = get_current_datetime_impl(timezone="Asia/Tokyo", format="japanese")
display(JSON(result))

In [ ]:
# Test 3: Equipment Status (工場向け)
print("=== Equipment Status Tool ===")

equipment_ids = ["LINE-A-01", "ROBOT-B-03", "PRESS-C-05"]

for eq_id in equipment_ids:
    result = get_equipment_status_impl(eq_id, include_history=False)
    print(f"\n{eq_id}:")
    print(f"  Status: {result.get('status', 'N/A')}")
    print(f"  Uptime: {result.get('uptime_hours', 0)}h")
    print(f"  Production: {result.get('production_count', 0)}")

## 4. Mock Agent Testing

Function Calling Agentの動作確認（Mock版）。

In [ ]:
agent = MockFunctionCallingAgent()

# Test 1: 単一ツール
query1 = "現在時刻を教えてください"
print(f"Query: {query1}")
answer1 = agent.run(query1, tools=["get_current_datetime"])
print(f"Answer: {answer1}\n")
agent.reset()

In [ ]:
# Test 2: 並列実行（重要）
query2 = "現在時刻と、100の2乗を計算して教えてください"
print(f"Query: {query2}")
answer2 = agent.run(query2, tools=["get_current_datetime", "calculate"])
print(f"Answer: {answer2}\n")
agent.reset()

In [ ]:
# Test 3: 工場向けツール
query3 = "LINE-A-01の設備状態を確認してください"
print(f"Query: {query3}")
answer3 = agent.run(query3, tools=["get_equipment_status"])
print(f"Answer: {answer3}\n")
agent.reset()

## 5. Custom Tool Design Exercise

独自ツールの設計演習（工場向けカスタマイズ）。

In [ ]:
# 保全スケジュールツール設計例
maintenance_schedule_schema = {
    "type": "function",
    "function": {
        "name": "get_maintenance_schedule",
        "description": "設備の保全スケジュールを取得します。定期保全、予防保全、修理履歴を確認できます。",
        "parameters": {
            "type": "object",
            "properties": {
                "equipment_id": {
                    "type": "string",
                    "description": "設備ID"
                },
                "period_days": {
                    "type": "integer",
                    "description": "取得期間（日数）",
                    "default": 30,
                    "minimum": 1,
                    "maximum": 365
                },
                "include_completed": {
                    "type": "boolean",
                    "description": "完了済み保全を含めるか",
                    "default": False
                }
            },
            "required": ["equipment_id"]
        }
    }
}

print("=== Custom Tool Schema ===")
display(JSON(maintenance_schedule_schema))

## Summary

### Day 17達成状況

✅ **完了項目**:
- Function Calling基礎理解
- ツール定義スキーマ作成（4ツール）
- ツール実装（4ツール）
- Mock Agent動作確認
- 並列実行検証

⬜ **Next Steps (Day 18)**:
- Azure AI Foundry上でAssistant作成
- Thread/Run/Message API実装
- Code Interpreter統合
- File Search統合